## `fastplotlib` demo

In [1]:
import numpy as np
import zmq
import fastplotlib as fpl

### Setup zmq subscriber client

In [2]:
context = zmq.Context()
sub = context.socket(zmq.SUB)
sub.setsockopt(zmq.SUBSCRIBE, b"")

# keep only the most recent message
sub.setsockopt(zmq.CONFLATE, 1)

# address must match publisher in Processor actor
sub.connect("tcp://127.0.0.1:5555")

<SocketContext(connect='tcp://127.0.0.1:5555')>

In [3]:
def get_buffer():
    """
    Gets the buffer from the publisher
    """
    try:
        b = sub.recv(zmq.NOBLOCK)
    except zmq.Again:
        pass
    else:
        return b
    
    return None

In [4]:
plot = fpl.Plot()

# initialize line plot with pre-allocated with cosine
xs = np.linspace(-10, 10, 100)
ys = np.cos(xs)

plot.add_line(data=np.dstack((xs, ys))[0], name="line")

def update_frame(p):
    # recieve memory with buffer
    buff = get_buffer()
    
    if buff is not None:
        # numpy array from buffer
        a = np.frombuffer(buff, dtype=np.float64)
        ix = a[-1]
        # set graphic data
        p["line"].data = a[:-1].reshape(100, 2)
        p.set_title(f"frame: {ix}")
        # change graphic color
        if ix % 2 == 0:
            p["line"].colors = "blue"
        else:
            p["line"].colors = "green"

plot.add_animations(update_frame)
              
plot.show()

RFBOutputContext()

No config found!
EGL says it can present to the window but not natively
/home/clewis7/venvs/improv/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

## `fastplotlib` is non-blocking!

In [ ]:
plot.canvas.get_stats()